In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bb-rates/BB_Rates_15min_from_10-2023.xlsx
/kaggle/input/bb-rates/BB_Rates_15min_from_10-2023.xlsx


In [39]:
import matplotlib.pyplot as plt

# Meeting 03.05.2024 Christoph-Nils-Arne
Varianz vorhersagen ist der Kern

einfaches Regressionsmodell 

GARCH
Nils LSTM

Kaggle aufsetzen 
Copilot

### Packages:
- AEON 
- SK Time 
- DARts 

optimal stopping 

LIteratur:
- https://rdcu.be/dGI38

subset = EURUSD[(EURUSD.index >= pd.to_datetime('2023-10-04')) & (EURUSD.index < pd.to_datetime('2023-10-05'))]

# Anzeigen des extrahierten Subsets# Nils - Arne 16.05.24
- extract USD - EURO pair from the dataset
- LinearRegression (ARne)
- GARCH(Nils)



In [40]:
!pip install arch --target=/kaggle/working/
import pandas as pd
import numpy as np
from matplotlib import pyplot
from arch import arch_model
file_path = '/kaggle/input/bb-rates/BB_Rates_15min_from_10-2023.xlsx'
data = pd.read_excel(file_path)


  Using cached arch-7.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached statsmodels-0.14.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.2 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached patsy-0.5.6-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached arch-7.0.0-cp310-cp310-manylinux_2_17_x86_64.manyli

In [41]:
start_index = 3
EURUSD = data.iloc[start_index:, :2]

# Rename the columns
EURUSD.columns = ['Dates', 'Close']

# Display the first few rows of the new dataset to verify
print(EURUSD.tail())

                     Dates   Close
13539  2024-04-17 09:30:00  1.0632
13540  2024-04-17 09:45:00  1.0625
13541  2024-04-17 10:00:00   1.063
13542  2024-04-17 10:15:00  1.0642
13543  2024-04-17 10:30:00  1.0647


In [ ]:
(13541/4)/24

141.05208333333334

In [ ]:
print(EURUSD.iloc[0,0])
print(EURUSD.iloc[1,0])
print(EURUSD.iloc[13540,0] - EURUSD.iloc[0,0])

2023-10-03 23:00:00
2023-10-03 23:15:00
196 days, 11:30:00


In [ ]:
85500/60

1425.0

In [ ]:
# Convert the 'Dates' column to datetime format
EURUSD['Dates'] = pd.to_datetime(EURUSD['Dates'])

# Calculate the difference between consecutive dates
time_diffs = EURUSD['Dates'].diff().dropna()

# Check if all differences are 15 minutes
is_15min_apart = (time_diffs == pd.Timedelta(minutes=15)).all()

print("Are all dates 15 minutes apart in increasing order?", is_15min_apart)

Are all dates 15 minutes apart in increasing order? False


In [ ]:
#test_EURUSD['Dates'] = EURUSD['Dates'].dt.date()

In [ ]:
time_diffs_equal_15min = time_diffs == pd.Timedelta(minutes=15)

In [ ]:
sum(time_diffs_equal_15min == False)

48

In [ ]:
# Find the indices where the time differences are not 15 minutes
indices_not_15min = time_diffs_equal_15min[~time_diffs_equal_15min].index

# Print the indices
print("Indices where time differences are not 15 minutes apart:", indices_not_15min)

Indices where time differences are not 15 minutes apart: Index([  291,   780,  1272,  1764,  2256,  2747,  2749,  3241,  3243,  3732,
        3733,  4227,  4228,  4717,  5206,  5692,  5733,  5735,  5737,  5738,
        5739,  5740,  5741,  6125,  6525,  7016,  7508,  7999,  8493,  8494,
        8982,  8984,  8985,  9470,  9959, 10449, 10450, 10462, 10846, 11338,
       11341, 11828, 11829, 12318, 12320, 12808, 13296, 13297],
      dtype='int64')


In [ ]:
indices_not_15min[1:] - indices_not_15min[:-1]

Index([489, 492, 492, 492, 491,   2, 492,   2, 489,   1, 494,   1, 489, 489,
       486,  41,   2,   2,   1,   1,   1,   1, 384, 400, 491, 492, 491, 494,
         1, 488,   2,   1, 485, 489, 490,   1,  12, 384, 492,   3, 487,   1,
       489,   2, 488, 488,   1],
      dtype='int64')

In [ ]:
(492/4)/24

5.125

In [ ]:
EURUSD.head()

In [52]:
EURUSD['Dates'] = pd.to_datetime(EURUSD['Dates'])

# Extrahieren aller Datenpunkte, die zum 4. Oktober 2023 gehören
subset = EURUSD[EURUSD['Dates'].dt.date == pd.to_datetime('2023-10-04').date()]

In [55]:
days_full = pd.DataFrame(EURUSD)

In [61]:
days_full['Dates'] = pd.to_datetime(days_full['Dates'])

In [63]:
days_full['Dates'] = days_full['Dates'].dt.date

In [68]:


# Zählen Sie die Anzahl der Einträge pro Tag
entries_per_day = days_full.groupby('Dates').size()



In [70]:

full_days = entries_per_day[entries_per_day == 96].index
print(full_days)

Index([2023-10-04, 2023-10-05, 2023-10-09, 2023-10-10, 2023-10-11, 2023-10-12,
       2023-10-16, 2023-10-17, 2023-10-18, 2023-10-19,
       ...
       2024-04-01, 2024-04-02, 2024-04-03, 2024-04-04, 2024-04-08, 2024-04-09,
       2024-04-10, 2024-04-11, 2024-04-15, 2024-04-16],
      dtype='object', name='Dates', length=109)


In [83]:


# Filtern Sie den ursprünglichen DataFrame, um nur die vollen Tage zu behalten
full_days_df = days_full[days_full['Dates'].isin(full_days)]

print(full_days_df)

            Dates   Close
7      2023-10-04  1.0468
8      2023-10-04  1.0467
9      2023-10-04  1.0467
10     2023-10-04  1.0465
11     2023-10-04  1.0465
...           ...     ...
13496  2024-04-16  1.0619
13497  2024-04-16  1.0623
13498  2024-04-16  1.0618
13499  2024-04-16  1.0617
13500  2024-04-16  1.0617

[10464 rows x 2 columns]


In [84]:
len(full_days_df)
#full_days_df = full_days_df.set_index('Dates')

10464

In [85]:
daily_series = []

# Iterieren Sie über die vollen Tage und extrahieren Sie die Zeitreihen
for day in full_days:
    daily_data = full_days_df[full_days_df['Dates'] == day]['Close'].values
    daily_series.append(daily_data)

# Konvertieren Sie die Liste der Zeitreihen in ein numpy Array
daily_series_array = np.array(daily_series, dtype=np.float32)

print(daily_series_array)
full_days_df = full_days_df.set_index('Dates')

[[1.0468 1.0467 1.0467 ... 1.0506 1.0506 1.0507]
 [1.0508 1.0508 1.0506 ... 1.0549 1.0549 1.0549]
 [1.0552 1.0552 1.0555 ... 1.0566 1.0566 1.0566]
 ...
 [1.0741 1.0743 1.0743 ... 1.0727 1.0728 1.0727]
 [1.0642 1.0648 1.0651 ... 1.0625 1.0625 1.0625]
 [1.0626 1.0627 1.0627 ... 1.0618 1.0617 1.0617]]


In [86]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

# Berechnen Sie die Anzahl der Tage für jeden Satz
num_days = daily_series_array.shape[0]
train_end = int(num_days * train_size)
val_end = train_end + int(num_days * val_size)

# Aufteilen der Daten in Trainings-, Validierungs- und Testsätze
train_data = daily_series_array[:train_end]
val_data = daily_series_array[train_end:val_end]
test_data = daily_series_array[val_end:]

print("Trainingsdaten:", train_data.shape)
print("Validierungsdaten:", val_data.shape)
print("Testdaten:", test_data.shape)

Trainingsdaten: (65, 96)
Validierungsdaten: (21, 96)
Testdaten: (23, 96)


In [87]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.tensor(sample, dtype=torch.float32)

# Erstellen Sie Dataset-Objekte für Trainings-, Validierungs- und Testsätze
train_dataset = TimeSeriesDataset(train_data)
val_dataset = TimeSeriesDataset(val_data)
test_dataset = TimeSeriesDataset(test_data)

# Erstellen Sie DataLoader-Objekte
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Beispiel: Durchlaufen der Daten mit dem DataLoader
for batch in train_loader:
    print(batch)
    break


tensor([[1.0538, 1.0539, 1.0538,  ..., 1.0582, 1.0582, 1.0582],
        [1.0881, 1.0882, 1.0881,  ..., 1.0849, 1.0848, 1.0850],
        [1.0798, 1.0799, 1.0799,  ..., 1.0874, 1.0878, 1.0878],
        ...,
        [1.0916, 1.0916, 1.0919,  ..., 1.0885, 1.0885, 1.0886],
        [1.0524, 1.0525, 1.0521,  ..., 1.0560, 1.0560, 1.0560],
        [1.0886, 1.0884, 1.0882,  ..., 1.0846, 1.0847, 1.0847]])


In [88]:
import torch.nn as nn

class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(EncoderLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        _, (h_n, c_n) = self.lstm(x)
        # h_n: (num_layers, batch_size, hidden_size)
        # c_n: (num_layers, batch_size, hidden_size)
        return h_n, c_n
        
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(DecoderLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, h_n, c_n):
        # x: (batch_size, seq_len, input_size)
        # h_n: (num_layers, batch_size, hidden_size)
        # c_n: (num_layers, batch_size, hidden_size)
        output, _ = self.lstm(x, (h_n, c_n))
        # output: (batch_size, seq_len, hidden_size)
        output = self.fc(output)
        # output: (batch_size, seq_len, output_size)
        return output


In [89]:
class VAE_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, latent_space_dim, num_layers):
        super(VAE_LSTM, self).__init__()
        
        self.encoder = EncoderLSTM(input_size, hidden_size, num_layers)
        
        self.mu_layer = nn.Linear(hidden_size, latent_space_dim)
        self.logvar_layer = nn.Linear(hidden_size, latent_space_dim)
        
        self.z_layer = nn.Linear(latent_space_dim, hidden_size)
        
        self.decoder = DecoderLSTM(latent_space_dim, hidden_size, num_layers, input_size)
            
    def forward(self, x):
        h_n, c_n = self.encoder(x)
        
        mu = self.mu_layer(h_n[-1])  # Verwende den letzten hidden state als Eingabe für mu und logvar
        logvar = self.logvar_layer(h_n[-1])
        
        if self.training:
            sigma = torch.exp(0.5 * logvar)
            epsilon = torch.randn_like(sigma)
            z = mu + epsilon * sigma
        else:
            z = mu
        
        z = self.z_layer(z)
        z = z.unsqueeze(1).repeat(1, x.size(1), 1)  # Wiederhole z für jede Zeitschritt
        output = self.decoder(z, h_n, c_n)
        
        return output, mu, logvar


In [90]:
def vae_loss(recon_x, x, mu, logvar):
    # Reconstruction losses are calculated using Mean Squared Error (MSE) and 
    # summed over all elements and batch
    mse_loss = nn.MSELoss(reduction = 'sum')(recon_x, x)

    # See Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    kld_loss = 1/2*torch.sum(1+logvar -mu.pow(2) - logvar.exp())

    total_loss = mse_loss - kld_loss

    return total_loss, mse_loss, kld_loss

In [91]:
learning_rate= 0.001
input_size = 1
hidden_size = 64
latent_space_dim = 16
num_layers = 2
epochs = 25

In [93]:
def get_device(cuda_preference=True):
    """Gets pytorch device object. If cuda_preference=True and 
        cuda is available on your system, returns a cuda device.
    
    Args:
        cuda_preference: bool, default True
            Set to true if you would like to get a cuda device
            
    Returns: pytorch device object
            Pytorch device
    """
    
    print('cuda available:', torch.cuda.is_available(), 
          '; cudnn available:', torch.backends.cudnn.is_available(),
          '; num devices:', torch.cuda.device_count())
    
    use_cuda = False if not cuda_preference else torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    device_name = torch.cuda.get_device_name(device) if use_cuda else 'cpu'
    print('Using device', device_name)
    return device

In [94]:
! python --version

torch.cuda.is_available(), torch.backends.cudnn.is_available(), torch.cuda.device_count()

device = get_device()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

num_cpus = os.cpu_count()

print(f"cpu_count: {num_cpus}")

Python 3.10.13
cuda available: True ; cudnn available: True ; num devices: 2
Using device Tesla T4
Using device: cuda
cpu_count: 4


In [100]:
model = VAE_LSTM(input_size,hidden_size, latent_space_dim, num_layers)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
total_losses, mse_losses, kl_losses, test_total_losses, test_mse_losses, test_kl_losses = run_training(
        model, torch.optim.Adam(model.parameters(), lr=learning_rate), vae_loss, epochs, train_loader, test_loader)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


RuntimeError: shape '[24576, 1]' is invalid for input of size 256